In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


- 문장길이를 숫자로 반환 (칼럼 새로 만들어서)
  - 이를 통해 이상치 제거
  - percentile 이용
  - IQR 찾아서 weight 곱하고 (Q_3+IQR*w <= <= Q_1 - IQR * w), w=5
    - 25 ~ 75% 사이 데이터만 추출

- 경우의 수 나눠
  - spam - spam(TP), ham(FN)
  - ham - ham(FP), spam(TN)
  - 이를 통해 accuracy, precision, recall, f1 score 산출 가능
  - $accuracy = \frac{TN+TP}{TN+TP+FN+TP}$
  - $precision = \frac{TP}{FP+TP}$ 예측값이 기준: 내가 spam이라고 예측한 것 중에 맞은 개수
  - $recall = \frac{TP}{FN+TP}$ 처음값이 기준: 실제로 spam인것중에 맞은 개수
  - 우리는 precision만 가져가는걸로!

- 그래서 우리는 precision 몇%인지, spam에는 ~~단어가 있더라 결과를 내야됨

- Tfidf할떄 Scaler: MinMaxScaler
  - Tfidfvectorizer하고 spam 여부 T/F 칼럼 만들어서 T/F끼리 다 더해서 평균해줌 (T_mean, F_mean)
  - T_mean - F_mean 해주면 값 나오고 이거 행별로 matmul 시키면 점수 나옴
  - 이 점수를 MinMaxScaler로 정규화해서 [0,1]값으로


In [2]:
import sys, os
PATH = "/content/drive/MyDrive/01. Programming/Practical NLP/"
sys.path.append(PATH)

In [3]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/01. Programming/Practical NLP/']

In [4]:
from pmi import *

In [10]:
import pandas as pd

FILE_PATH = "drive/MyDrive/01. Programming/Practical NLP/data/Week 15/spam-ham.csv"
df = pd.read_csv(FILE_PATH, index_col=0)
pd.set_option('display.max_colwidth', None)
df.head(3)

,spam,text
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
dtm = tfidf.fit_transform(df.text).toarray()
dtm

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# spam, ham => T/F
mask = df.spam.astype(bool).values
mask

array([False, False,  True, ..., False, False, False])

In [13]:
spam_avg = dtm[mask].mean(axis=0)
ham_avg = dtm[~mask].mean(axis=0)

In [14]:
spamness = dtm.dot(spam_avg - ham_avg)

In [15]:
# scaling
from sklearn.preprocessing import MinMaxScaler
df['tfidf_score']=MinMaxScaler().fit_transform(spamness.reshape(-1, 1))

# .fit(), .transform() 따로 있음
df.head(3)

,spam,text,tfidf_score
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0.239998
1,0,Ok lar... Joking wif u oni...,0.216380
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,0.646709


In [16]:
df['prediction'] = (df['tfidf_score'] > 0.5).astype(int)
df.head(3)

,spam,text,tfidf_score,prediction
0,0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0.239998,0
1,0,Ok lar... Joking wif u oni...,0.216380,0
2,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,0.646709,1


In [17]:
from sklearn.metrics import confusion_matrix, precision_score, f1_score, recall_score
confusion_matrix(df.spam, df.prediction)

array([[4174,   25],
       [ 133,  505]])